In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.12.0


In [2]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [29]:
def preprocess(df, name):
    df_gyro = pd.DataFrame(columns=['time','GY(x)', 'GY(y)', 'GY(z)'])
    df_acc = pd.DataFrame(columns=['time','LA(x)', 'LA(y)', 'LA(z)'])
    
    df_gyro[['time','GY(x)', 'GY(y)', 'GY(z)']] = df[['TimeStamp','GY(x)', 'GY(y)', 'GY(z)']].copy()
#     df_gyro['name'] = df_gyro['name'].fillna(name)
    df_gyro = df_gyro.dropna()
    df_acc[['time','LA(x)', 'LA(y)', 'LA(z)']] = df[['TimeStamp','LA(x)', 'LA(y)', 'LA(z)']].copy()
#     df_acc['name'] = df_acc['name'].fillna(name)
    df_acc = df_acc.dropna()
    
    df_gyro['time'] = pd.to_datetime(df_gyro['time'], unit='ms')
    df_indexed_gyro = df_gyro.set_index('time')
    resampled_df_gyro = df_indexed_gyro.resample('5ms').mean().interpolate()
    resampled_df_gyro['name'] = name
    resampled_df_gyro.reset_index(inplace=True)
    
    df_acc['time'] = pd.to_datetime(df_acc['time'], unit='ms')
    df_indexed_acc = df_acc.set_index('time')
    resampled_df_acc = df_indexed_acc.resample('5ms').mean().interpolate()
#     resampled_df_acc['name'] = name
    resampled_df_acc.reset_index(inplace=True)
    
    # Inner join: Only the common values in both DataFrames are included in the merged DataFrame (inner_merged_df).
    
    inner_merged_df = pd.merge(resampled_df_acc, resampled_df_gyro, on='time', how='inner')
    
    return inner_merged_df

In [30]:
df_final = pd.DataFrame(columns=['name','time','LA(x)', 'LA(y)', 'LA(z)','GY(x)', 'GY(y)', 'GY(z)'])
directory = os.path.join(os.getcwd(), 'straightWalk')
x=1
firstTime = True
for filename in os.listdir(directory):
    if (filename.endswith(".csv")):
        filepath = os.path.join(directory, filename)
        df_original = pd.read_csv(filepath)
       
        df_processed = preprocess(df_original, filename[:-4])

        df_final = pd.concat([df_final, df_processed])
        
df_final

,name,time,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z)
0,alibadawy,1970-01-01 00:00:00.000,-0.268265,-0.005538,-0.071094,-0.016798,0.103233,0.033138
1,alibadawy,1970-01-01 00:00:00.005,-0.163861,-0.004340,0.122527,-0.017613,0.110563,0.034157
2,alibadawy,1970-01-01 00:00:00.010,-0.018824,-0.011455,0.059537,-0.018936,0.123697,0.031000
3,alibadawy,1970-01-01 00:00:00.015,0.043948,-0.010231,0.135187,-0.019445,0.136423,0.030288
4,alibadawy,1970-01-01 00:00:00.020,0.106720,-0.009007,0.210836,-0.013744,0.146603,0.027030
...,...,...,...,...,...,...,...,...
5275,zoz,1970-01-01 00:00:26.375,0.032646,0.028275,0.519662,0.135150,-0.022907,0.223494
5276,zoz,1970-01-01 00:00:26.380,-0.478584,-0.296353,0.069982,0.152254,-0.098041,0.268391
5277,zoz,1970-01-01 00:00:26.385,-0.897290,-0.563466,-0.287334,0.155308,-0.165234,0.318786
5278,zoz,1970-01-01 00:00:26.390,-1.035197,-0.655348,-0.474507,0.145534,-0.212269,0.362156


In [33]:
df_final.isnull().sum()

time     0
LA(x)    0
LA(y)    0
LA(z)    0
GY(x)    0
GY(y)    0
GY(z)    0
name     0
dtype: int64

In [37]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155302 entries, 0 to 5279
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   name    155302 non-null  object        
 1   time    155302 non-null  datetime64[ns]
 2   LA(x)   155302 non-null  float64       
 3   LA(y)   155302 non-null  float64       
 4   LA(z)   155302 non-null  float64       
 5   GY(x)   155302 non-null  float64       
 6   GY(y)   155302 non-null  float64       
 7   GY(z)   155302 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 10.7+ MB


In [35]:
df_final['name'].value_counts()

karen              6358
mayar              5916
habibatarabay      5835
yusuf              5644
hananabil          5560
waed               5534
farah              5492
moustafa           5417
rolla              5407
mohameddawoud      5387
zoz                5280
ola                5171
mohamedelshoura    5131
ghazaly            5092
nourhan            5069
seifana            4974
yaseen             4889
mira               4848
alibadawy          4836
hussenhany         4800
hashem             4795
youssefemad        4597
seifzakaria        4590
janah              4545
ismail             4522
moataz             4467
tokhye             4421
youssef            4249
salwa              4236
heidi              4125
dinaelgendy        4115
Name: name, dtype: int64

In [42]:
df_Xtime = df_final.drop(['time'], axis = 1).copy()
df_Xtime.shape

(155302, 7)

In [44]:
label = LabelEncoder()
df_Xtime['label'] = label.fit_transform(df_Xtime['name'])
df_Xtime

,name,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z),label
0,alibadawy,-0.268265,-0.005538,-0.071094,-0.016798,0.103233,0.033138,0
1,alibadawy,-0.163861,-0.004340,0.122527,-0.017613,0.110563,0.034157,0
2,alibadawy,-0.018824,-0.011455,0.059537,-0.018936,0.123697,0.031000,0
3,alibadawy,0.043948,-0.010231,0.135187,-0.019445,0.136423,0.030288,0
4,alibadawy,0.106720,-0.009007,0.210836,-0.013744,0.146603,0.027030,0
...,...,...,...,...,...,...,...,...
5275,zoz,0.032646,0.028275,0.519662,0.135150,-0.022907,0.223494,30
5276,zoz,-0.478584,-0.296353,0.069982,0.152254,-0.098041,0.268391,30
5277,zoz,-0.897290,-0.563466,-0.287334,0.155308,-0.165234,0.318786,30
5278,zoz,-1.035197,-0.655348,-0.474507,0.145534,-0.212269,0.362156,30


In [45]:
label.classes_

array(['alibadawy', 'dinaelgendy', 'farah', 'ghazaly', 'habibatarabay',
       'hananabil', 'hashem', 'heidi', 'hussenhany', 'ismail', 'janah',
       'karen', 'mayar', 'mira', 'moataz', 'mohameddawoud',
       'mohamedelshoura', 'moustafa', 'nourhan', 'ola', 'rolla', 'salwa',
       'seifana', 'seifzakaria', 'tokhye', 'waed', 'yaseen', 'youssef',
       'youssefemad', 'yusuf', 'zoz'], dtype=object)

In [46]:
X = df_Xtime[['LA(x)', 'LA(y)', 'LA(z)','GY(x)', 'GY(y)', 'GY(z)']].copy()
y = df_Xtime['label'].copy()

In [53]:
X

,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z)
0,-0.268265,-0.005538,-0.071094,-0.016798,0.103233,0.033138
1,-0.163861,-0.004340,0.122527,-0.017613,0.110563,0.034157
2,-0.018824,-0.011455,0.059537,-0.018936,0.123697,0.031000
3,0.043948,-0.010231,0.135187,-0.019445,0.136423,0.030288
4,0.106720,-0.009007,0.210836,-0.013744,0.146603,0.027030
...,...,...,...,...,...,...
5275,0.032646,0.028275,0.519662,0.135150,-0.022907,0.223494
5276,-0.478584,-0.296353,0.069982,0.152254,-0.098041,0.268391
5277,-0.897290,-0.563466,-0.287334,0.155308,-0.165234,0.318786
5278,-1.035197,-0.655348,-0.474507,0.145534,-0.212269,0.362156


In [60]:
scaler = StandardScaler()
X_transformed = X.copy()
X_transformed = scaler.fit_transform(X_transformed)

scaled_X = pd.DataFrame(data = X_transformed, columns = ['LA(x)', 'LA(y)', 'LA(z)','GY(x)', 'GY(y)', 'GY(z)'])
scaled_X['label'] = y.values

scaled_X




,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z),label
0,-0.157996,0.116219,-0.109081,-0.070382,0.176505,0.142357,0
1,-0.120536,0.116679,-0.048151,-0.071206,0.181998,0.143630,0
2,-0.068498,0.113950,-0.067973,-0.072545,0.191841,0.139684,0
3,-0.045976,0.114420,-0.044167,-0.073060,0.201378,0.138793,0
4,-0.023454,0.114889,-0.020362,-0.067292,0.209007,0.134721,0
...,...,...,...,...,...,...,...
155297,-0.050031,0.129187,0.076822,0.083323,0.081973,0.380324,30
155298,-0.233456,0.004690,-0.064686,0.100625,0.025666,0.436450,30
155299,-0.383684,-0.097750,-0.177129,0.103714,-0.024689,0.499450,30
155300,-0.433164,-0.132987,-0.236030,0.093828,-0.059938,0.553668,30


In [73]:
import scipy.stats as stats

Fs = 200
frame_size = Fs*2 # 800
hop_size = Fs*2 # 800 //NO_overlapping

In [99]:
def get_frames(df, frame_size, hop_size):
    N_FEATURES = 6

    frames = []
    labels = []
    
    last_value = df['label'].iloc[-1]
    for j in range (0,last_value+1): 
        filtered_df = df[df['label'] == j]
        filtered_df = filtered_df.drop('label', axis = 1)
        for i in range(0, len(filtered_df) - frame_size, hop_size):
            df_frame = filtered_df[i: i + frame_size].copy()
            
            

            label = j

            temp_frames = df_frame.to_numpy()
            
            frames.append(temp_frames)
            labels.append(label)

    frames = np.asarray(frames)
    labels = np.asarray(labels)
    
    return frames, labels
X, y = get_frames(scaled_X,frame_size, hop_size)
X.shape, y.shape
        

((372, 400, 6), (372,))

In [101]:
X

array([[[-0.15799562,  0.1162193 , -0.10908088, -0.07038199,
          0.17650491,  0.14235708],
        [-0.12053648,  0.11667867, -0.04815117, -0.07120587,
          0.18199827,  0.1436298 ],
        [-0.0684984 ,  0.11395025, -0.06797312, -0.07254468,
          0.19184054,  0.13968437],
        ...,
        [-1.27402103,  1.38117851,  0.38645973,  0.73290068,
         -0.48733338,  0.57609947],
        [-1.17607806,  1.26148323,  0.15098442,  0.75082007,
         -0.48069558,  0.65208075],
        [-1.05377589,  1.1031733 , -0.12009711,  0.75174692,
         -0.49420008,  0.71355301]],

       [[-0.89566728,  0.91073417, -0.39221972,  0.73691706,
         -0.52761803,  0.75364366],
        [-0.70234856,  0.65927213, -0.68947242,  0.69520814,
         -0.57614272,  0.77464349],
        [-0.48523541,  0.35411074, -1.06773581,  0.60962769,
         -0.62237851,  0.7723526 ],
        ...,
        [-1.55826157, -1.66461684,  0.36107435, -1.25221157,
          0.87187278, -1.87105139],
  

In [102]:
y

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18,
       18, 18, 18, 18, 18

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)


In [104]:
X_train.shape, X_test.shape

((260, 400, 6), (112, 400, 6))

In [105]:
# Get the unique elements and their counts
unique_elements, counts = np.unique(y_test, return_counts=True)

# Print the unique elements and their counts
for element, count in zip(unique_elements, counts):
    print(f"Number {element} is repeated {count} times")

Number 0 is repeated 4 times
Number 1 is repeated 3 times
Number 2 is repeated 4 times
Number 3 is repeated 4 times
Number 4 is repeated 4 times
Number 5 is repeated 4 times
Number 6 is repeated 3 times
Number 7 is repeated 3 times
Number 8 is repeated 3 times
Number 9 is repeated 3 times
Number 10 is repeated 3 times
Number 11 is repeated 4 times
Number 12 is repeated 4 times
Number 13 is repeated 4 times
Number 14 is repeated 3 times
Number 15 is repeated 4 times
Number 16 is repeated 4 times
Number 17 is repeated 4 times
Number 18 is repeated 4 times
Number 19 is repeated 4 times
Number 20 is repeated 4 times
Number 21 is repeated 3 times
Number 22 is repeated 4 times
Number 23 is repeated 3 times
Number 24 is repeated 3 times
Number 25 is repeated 4 times
Number 26 is repeated 4 times
Number 27 is repeated 3 times
Number 28 is repeated 3 times
Number 29 is repeated 4 times
Number 30 is repeated 4 times


In [106]:
y_test

array([25,  5, 28, 29, 21, 27, 11,  3, 20,  0, 23,  4,  7, 30, 11, 21, 18,
       26, 14, 24, 11, 12,  5, 27,  7, 17,  3, 29, 18, 15, 24, 20, 13, 17,
       14,  2, 25,  0, 23,  6,  9,  9, 22,  8, 17, 14, 13, 30,  2, 15,  3,
       15,  4,  9, 16, 13, 10, 11, 26, 28,  6,  8, 27,  4,  1, 16,  0, 16,
        2, 18, 12, 16, 13, 22, 30, 19, 19, 17, 26, 20,  3, 29,  1,  0, 30,
       25,  4, 23, 12, 26,  6,  2, 10, 22,  8,  5, 21, 15, 20, 19, 28, 25,
       18, 29,  7,  1,  5, 19, 24, 22, 12, 10])

In [107]:
X_train.shape, X_test.shape

((260, 400, 6), (112, 400, 6))

In [109]:
X_train[0].shape, X_test[0].shape

((400, 6), (400, 6))

In [110]:
train_shape_0 = X_train.shape[0]
test_shape_0 = X_test.shape[0]
train_shape_1 = X_train.shape[1]
test_shape_1 = X_test.shape[1]
train_shape_1
X_train = X_train.reshape(train_shape_0, train_shape_1, 6, 1)
X_test = X_test.reshape(test_shape_0, test_shape_1, 6, 1)

X_train[0].shape, X_test[0].shape

((400, 6, 1), (400, 6, 1))

In [111]:
model = Sequential()
model.add(Conv2D(50, (2,2), activation = 'relu', input_shape = X_train[0].shape))
model.add(MaxPooling2D(pool_size=(2,1)))
model.add(Dropout(0.1))

model.add(Conv2D(100, (2,2), activation = 'relu', input_shape = X_train[0].shape))
model.add(MaxPooling2D(pool_size=(2,1)))
model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(31, activation='softmax'))

In [113]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 10, validation_data= (X_test, y_test), verbose=1)

Epoch 1/10
9/9 [==============================] - 2s 100ms/step - loss: 3.4299 - accuracy: 0.0269 - val_loss: 3.4307 - val_accuracy: 0.0357
Epoch 2/10
9/9 [==============================] - 1s 87ms/step - loss: 3.4296 - accuracy: 0.0423 - val_loss: 3.4307 - val_accuracy: 0.0357
Epoch 3/10
9/9 [==============================] - 1s 87ms/step - loss: 3.4295 - accuracy: 0.0423 - val_loss: 3.4307 - val_accuracy: 0.0357
Epoch 4/10
9/9 [==============================] - 1s 86ms/step - loss: 3.4295 - accuracy: 0.0423 - val_loss: 3.4307 - val_accuracy: 0.0357
Epoch 5/10
9/9 [==============================] - 1s 87ms/step - loss: 3.4295 - accuracy: 0.0423 - val_loss: 3.4307 - val_accuracy: 0.0357
Epoch 6/10
9/9 [==============================] - 1s 88ms/step - loss: 3.4294 - accuracy: 0.0423 - val_loss: 3.4307 - val_accuracy: 0.0357
Epoch 7/10
9/9 [==============================] - 1s 87ms/step - loss: 3.4295 - accuracy: 0.0423 - val_loss: 3.4307 - val_accuracy: 0.0357
Epoch 8/10
9/9 [==========